# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

from livelossplot import PlotLosses

# User Settings

In [2]:
import pickle

# class Company(object):
#     def __init__(self, name, value):
#         self.name = name
#         self.value = value

# with open('company_data.pkl', 'wb') as output:
#     company1 = Company('banana', 40)
#     pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)

#     company2 = Company('spam', 42)
#     pickle.dump(company2, output, pickle.HIGHEST_PROTOCOL)

# del company1
# del company2

# with open('company_data.pkl', 'rb') as input:
#     company1 = pickle.load(input)
#     print(company1.name)  # -> banana
#     print(company1.value)  # -> 40

#     company2 = pickle.load(input)
#     print(company2.name) # -> spam
#     print(company2.value)  # -> 42

    
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# sample usage
#save_object(company1, 'company1.pkl')

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = False
save_masks = False
print_loss = True
output_tensors = False
train_flag = True
EPOCHS = 1
iteration = "-2"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 4
num_batch = 10000 #25000 #11112
imageSize = 416
SAVE_LOCATION = "./data/models/model_running"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
#val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
#val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    best_model = None
    best_loss = math.inf
    #model.train()  
    liveloss = PlotLosses()
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    batch_log = logly.logly(True, num_epochs, num_batch)
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
        
        #Training    
            
        for batch in range(num_batch):
            epoch_loss = 0
            running_loss = 0.0
            running_corrects = 0
            logs = {}
            
            if print_loss:
                #print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                #dtype=torch.int64
                ### Why are we using float?
                #labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize, dtype = torch.float32).to(device)
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                #print("loading tensor:", i,  loc, flush=True)
                #print("tensor max:", torch.max(labels[i]))
                i += 1

            if train_flag:
                phase = 'train'
                #print("Setting model in training mode")
                # Set model to training mode
                model.train() 
            else:
                phase = 'val'
                #print("Setting model in validation mode")
                # Set model to evaluate mode
                model.eval()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                #outputs = model(inputs.to(device)).to(device)
                #outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss = loss.item()

            outputs = sigmoid_function(outputs)
        # Validation
    #         _, preds = torch.max(outputs, 1)
    #         running_loss += loss.detach() * inputs.size(0)
    #         running_corrects += torch.sum(preds == labels.data, dtype = torch.float32)
    #         labels = labels.to(dtype=torch.long)
    #         running_corrects += torch.sum(preds == labels.data)

    #         e_loss = running_loss / len(dataloaders.dataset)
    #         epoch_acc = running_corrects.double() / len(dataloaders.dataset)

    #         print("Running corrects: {}".format(running_corrects), flush=True)


            #logs['accuracy'] = epoch_acc

            batch_log.log(phase, epoch, batch, epoch_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)

            if show_live_loss and phase == 'train':
                logs['log loss'] = epoch_loss
                logs['accuracy'] = 1 - epoch_loss
                liveloss.update(logs)
                liveloss.draw()

            if save_masks and phase == 'val':
                # need to fix before running
                outputs_dir = "data/outputs/results/" + str(datetime.now().date())
                os.makedirs(outputs_dir, exist_ok=True)
                threshold = 0.05
                j = 0
                for locations in paths:
                    img = Image.open(locations)
                    #num = locations.replace("./data/validation/photos/", "").replace(".png", "")
                    num = locations.replace("./data/validation/photos/", "").replace(".png", "")
                    for threshold in [0.05, .25, .5, .75]:
                        img = data_factory.showMaskOnImage(img,outputs[j], threshold)
                        img.save(outputs_dir  + "/val-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
                    j += 1

            if print_loss:
                print("loss: {}".format(epoch_loss), flush=True)
                print('---------------', flush=True)
                #input("please enter any key to continue")

            if output_tensors:
                return inputs, outputs, paths

            #save best copy of  model
            #if phase == 'val' and epoch_loss < best_loss:
            if phase == 'train' and epoch_loss < best_loss:
                best_loss = epoch_loss
                torch.save(model, LOAD_LOCATION + iteration)
                torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
                #torch.save(model, SAVE_LOCATION)
                #torch.save(model, SAVE_LOCATION.replace("model", "model_best"))
        
    time_elapsed = time.time() - since
    
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    log_name = str(datetime.now().date()) + iteration
    batch_log.write(log_dir, log_name)
    
    #logs_dir = "data/logs/" + str(datetime.now().date())
    #save_object(log, logs_dir)
    
    #completed model
    torch.save(model,SAVE_LOCATION)
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet16(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
    print('initializing model with random weights', flush=True)
    torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)

    torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  


#model = model.to(device)

loading weights from ./data/models/model_best


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/1
Batch 1/10000
loss: 0.15400280058383942
---------------
Batch 2/10000
loss: 0.39508771896362305
---------------
Batch 3/10000
loss: 0.3525592088699341
---------------
Batch 4/10000
loss: 0.2308076024055481
---------------
Batch 5/10000
loss: 0.21460574865341187
---------------
Batch 6/10000
loss: 0.10932700335979462
---------------
Batch 7/10000
loss: 0.3484956920146942
---------------
Batch 8/10000
loss: 0.2039801925420761
---------------
Batch 9/10000
loss: 0.2503811717033386
---------------
Batch 10/10000
loss: 0.17669028043746948
---------------
Batch 11/10000
loss: 0.26877889037132263
---------------
Batch 12/10000
loss: 0.24517273902893066
---------------
Batch 13/10000
loss: 0.14211298525333405
---------------
Batch 14/10000
loss: 0.1989617496728897
---------------
Batch 15/10000
loss: 0.14593631029129028
---------------
Batch 16/10000
loss: 0.19577713310718536
---------------
Batch 17/10000
loss: 0.11504247039556503
---------------
Batch 18/10000
loss: 0.1948026865720

---------------
Batch 145/10000
loss: 0.16319788992404938
---------------
Batch 146/10000
loss: 0.11529023200273514
---------------
Batch 147/10000
loss: 0.1433493196964264
---------------
Batch 148/10000
loss: 0.22357814013957977
---------------
Batch 149/10000
loss: 0.25634336471557617
---------------
Batch 150/10000
loss: 0.17743220925331116
---------------
Batch 151/10000
loss: 0.1400173157453537
---------------
Batch 152/10000
loss: 0.1635008007287979
---------------
Batch 153/10000
loss: 0.22675061225891113
---------------
Batch 154/10000
loss: 0.2222857028245926
---------------
Batch 155/10000
loss: 0.20871005952358246
---------------
Batch 156/10000
loss: 0.14954286813735962
---------------
Batch 157/10000
loss: 0.1516743004322052
---------------
Batch 158/10000
loss: 0.22535625100135803
---------------
Batch 159/10000
loss: 0.2972717881202698
---------------
Batch 160/10000
loss: 0.23613861203193665
---------------
Batch 161/10000
loss: 0.17964088916778564
---------------
Batc

Batch 287/10000
loss: 0.12513163685798645
---------------
Batch 288/10000
loss: 0.10110196471214294
---------------
Batch 289/10000
loss: 0.1342429369688034
---------------
Batch 290/10000
loss: 0.15871018171310425
---------------
Batch 291/10000
loss: 0.2571951150894165
---------------
Batch 292/10000
loss: 0.14675746858119965
---------------
Batch 293/10000
loss: 0.1537984162569046
---------------
Batch 294/10000
loss: 0.15968884527683258
---------------
Batch 295/10000
loss: 0.13116508722305298
---------------
Batch 296/10000
loss: 0.12820635735988617
---------------
Batch 297/10000
loss: 0.2421235740184784
---------------
Batch 298/10000
loss: 0.12714870274066925
---------------
Batch 299/10000
loss: 0.2476484775543213
---------------
Batch 300/10000
loss: 0.19915758073329926
---------------
Batch 301/10000
loss: 0.17826399207115173
---------------
Batch 302/10000
loss: 0.30118128657341003
---------------
Batch 303/10000
loss: 0.2445160448551178
---------------
Batch 304/10000
loss

---------------
Batch 430/10000
loss: 0.22235238552093506
---------------
Batch 431/10000
loss: 0.18882840871810913
---------------
Batch 432/10000
loss: 0.18858113884925842
---------------
Batch 433/10000
loss: 0.18206752836704254
---------------
Batch 434/10000
loss: 0.23898649215698242
---------------
Batch 435/10000
loss: 0.21288196742534637
---------------
Batch 436/10000
loss: 0.41731882095336914
---------------
Batch 437/10000
loss: 0.15776926279067993
---------------
Batch 438/10000
loss: 0.3946739137172699
---------------
Batch 439/10000
loss: 0.21474777162075043
---------------
Batch 440/10000
loss: 0.22233426570892334
---------------
Batch 441/10000
loss: 0.26482951641082764
---------------
Batch 442/10000
loss: 0.2703961431980133
---------------
Batch 443/10000
loss: 0.22841699421405792
---------------
Batch 444/10000
loss: 0.26994964480400085
---------------
Batch 445/10000
loss: 0.2360333353281021
---------------
Batch 446/10000
loss: 0.1871214061975479
---------------
Ba

Batch 572/10000
loss: 0.21906836330890656
---------------
Batch 573/10000
loss: 0.20031528174877167
---------------
Batch 574/10000
loss: 0.23330801725387573
---------------
Batch 575/10000
loss: 0.2555988132953644
---------------
Batch 576/10000
loss: 0.39401596784591675
---------------
Batch 577/10000
loss: 0.26834577322006226
---------------
Batch 578/10000
loss: 0.22203625738620758
---------------
Batch 579/10000
loss: 0.11753983050584793
---------------
Batch 580/10000
loss: 0.16783377528190613
---------------
Batch 581/10000
loss: 0.18823306262493134
---------------
Batch 582/10000
loss: 0.2407529503107071
---------------
Batch 583/10000
loss: 0.22728769481182098
---------------
Batch 584/10000
loss: 0.25377926230430603
---------------
Batch 585/10000
loss: 0.22419585287570953
---------------
Batch 586/10000
loss: 0.17194892466068268
---------------
Batch 587/10000
loss: 0.12413905560970306
---------------
Batch 588/10000
loss: 0.15604232251644135
---------------
Batch 589/10000


...

---------------
Batch 9319/10000
loss: 0.6454257369041443
---------------
Batch 9320/10000
loss: 0.6358826160430908
---------------
Batch 9321/10000
loss: 0.6171776652336121
---------------
Batch 9322/10000
loss: 0.6639617681503296
---------------
Batch 9323/10000
loss: 0.6268901824951172
---------------
Batch 9324/10000
loss: 0.5936596989631653
---------------
Batch 9325/10000
loss: 0.6358146667480469
---------------
Batch 9326/10000
loss: 0.6234488487243652
---------------
Batch 9327/10000
loss: 0.614841878414154
---------------
Batch 9328/10000
loss: 0.5706757307052612
---------------
Batch 9329/10000
loss: 0.6108506917953491
---------------
Batch 9330/10000
loss: 0.6576794385910034
---------------
Batch 9331/10000
loss: 0.5878835916519165
---------------
Batch 9332/10000
loss: 0.584983766078949
---------------
Batch 9333/10000
loss: 0.6341996192932129
---------------
Batch 9334/10000
loss: 0.626394510269165
---------------
Batch 9335/10000
loss: 0.6058877110481262
---------------
B

---------------
Batch 9461/10000
loss: 0.5888103246688843
---------------
Batch 9462/10000
loss: 0.639113187789917
---------------
Batch 9463/10000
loss: 0.5758923888206482
---------------
Batch 9464/10000
loss: 0.6291313767433167
---------------
Batch 9465/10000
loss: 0.5656988620758057
---------------
Batch 9466/10000
loss: 0.65692138671875
---------------
Batch 9467/10000
loss: 0.7260209321975708
---------------
Batch 9468/10000
loss: 0.6105960607528687
---------------
Batch 9469/10000
loss: 0.697503387928009
---------------
Batch 9470/10000
loss: 0.6618523597717285
---------------
Batch 9471/10000
loss: 0.6540804505348206
---------------
Batch 9472/10000
loss: 0.5689187049865723
---------------
Batch 9473/10000
loss: 0.6135716438293457
---------------
Batch 9474/10000
loss: 0.5994933247566223
---------------
Batch 9475/10000
loss: 0.6200462579727173
---------------
Batch 9476/10000
loss: 0.6485520601272583
---------------
Batch 9477/10000
loss: 0.5457866787910461
---------------
Ba

---------------
Batch 9603/10000
loss: 0.6168246865272522
---------------
Batch 9604/10000
loss: 0.6386718153953552
---------------
Batch 9605/10000
loss: 0.6873226165771484
---------------
Batch 9606/10000
loss: 0.6305270195007324
---------------
Batch 9607/10000
loss: 0.6615278124809265
---------------
Batch 9608/10000
loss: 0.6128313541412354
---------------
Batch 9609/10000
loss: 0.6729503870010376
---------------
Batch 9610/10000
loss: 0.6470738649368286
---------------
Batch 9611/10000
loss: 0.539701521396637
---------------
Batch 9612/10000
loss: 0.6194424033164978
---------------
Batch 9613/10000
loss: 0.6387487649917603
---------------
Batch 9614/10000
loss: 0.5934211015701294
---------------
Batch 9615/10000
loss: 0.5933173894882202
---------------
Batch 9616/10000
loss: 0.6147316098213196
---------------
Batch 9617/10000
loss: 0.5748358368873596
---------------
Batch 9618/10000
loss: 0.6517873406410217
---------------
Batch 9619/10000
loss: 0.6211022734642029
---------------

loss: 0.6455614566802979
---------------
Batch 9745/10000
loss: 0.5860134959220886
---------------
Batch 9746/10000
loss: 0.618128776550293
---------------
Batch 9747/10000
loss: 0.6247801780700684
---------------
Batch 9748/10000
loss: 0.6722739338874817
---------------
Batch 9749/10000
loss: 0.6374333500862122
---------------
Batch 9750/10000
loss: 0.6000521779060364
---------------
Batch 9751/10000
loss: 0.6325792670249939
---------------
Batch 9752/10000
loss: 0.6030538082122803
---------------
Batch 9753/10000
loss: 0.6678194999694824
---------------
Batch 9754/10000
loss: 0.6087977290153503
---------------
Batch 9755/10000
loss: 0.6690203547477722
---------------
Batch 9756/10000
loss: 0.6896595358848572
---------------
Batch 9757/10000
loss: 0.6435112357139587
---------------
Batch 9758/10000
loss: 0.48830264806747437
---------------
Batch 9759/10000
loss: 0.6138619780540466
---------------
Batch 9760/10000
loss: 0.6722291111946106
---------------
Batch 9761/10000
loss: 0.585750

Batch 9886/10000
loss: 0.5655587911605835
---------------
Batch 9887/10000
loss: 0.5882012248039246
---------------
Batch 9888/10000
loss: 0.5884173512458801
---------------
Batch 9889/10000
loss: 0.6075550317764282
---------------
Batch 9890/10000
loss: 0.6804261803627014
---------------
Batch 9891/10000
loss: 0.6312990784645081
---------------
Batch 9892/10000
loss: 0.6493174433708191
---------------
Batch 9893/10000
loss: 0.555778443813324
---------------
Batch 9894/10000
loss: 0.665469229221344
---------------
Batch 9895/10000
loss: 0.578748345375061
---------------
Batch 9896/10000
loss: 0.6324713230133057
---------------
Batch 9897/10000
loss: 0.6126642227172852
---------------
Batch 9898/10000
loss: 0.6970215439796448
---------------
Batch 9899/10000
loss: 0.5870174765586853
---------------
Batch 9900/10000
loss: 0.6250601410865784
---------------
Batch 9901/10000
loss: 0.6466280221939087
---------------
Batch 9902/10000
loss: 0.5800870060920715
---------------
Batch 9903/10000


# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [11]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

In [ ]:
outputs[0]

In [12]:
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)

/data/photos/999.png
999


In [2]:
import sys
print(sys.version)

3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]
